# Langkah #5 - Pengujian Hipotesis
---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('insurance.csv')
df.head()

---
### Hipotesis #1. Tagihan kesehatan perokok lebih tinggi daripada tagihan kesehatan non perokok

**Langkah 1: Siapkan hipotesis**
- H0: Perokok memiliki tagihan kesehatan > daripada tagihan non perokok
- H1: Perokok memiliki tagihan kesehatan <= daripada tagihan non perokok
- $\alpha$: 5%

In [ ]:
# Tentukan populasinya
# 1. Sampel Tagihan Perokok
jml_perokok = df[df['smoker']=='yes'].value_counts().sum()
print("Jumlah perokok: {} orang".format(jml_perokok))

# 2. Sampel Tagihan non perokok
jml_non_perokok = df[df['smoker']=='no'].value_counts().sum()
print("Jumlah non perokok: {} orang".format(jml_non_perokok))
print("Jumlah keseluruhan sampel: {}".format(df.shape[0]))

# Tentukan rata-rata charges(tagihan) dari kedua sampel
print("\nRata-rata tagihan pada perokok dan non perokok: ")
mu = df[['charges']].groupby(df['smoker']).mean()
print(mu)
mu1 = mu['charges']['no']
mu2 = mu['charges']['yes']

In [ ]:
# Plot distribusi charges di seluruh populasi
sns.displot(df, x='charges')
plt.axvline(x=mu1, linestyle='--', c='red', label='mean tagihan non perokok')
plt.axvline(x=mu2, linestyle='--', c='purple', label='mean tagihan perokok')

Berdasarkan grafik di atas terlihat bahwa bentuk distribusi data adalah positive-skew. Maka dari itu kita perlu menormalkan distribusi charges dengan mengeluarkan data outlier dari distribusi. Dari grafik terlihat bahwa outlier dimulai dari data `charges` dengan nilai di atas 15K. 

In [ ]:
# Filter data tagihan, gunakan tagihan (charges) < 15K
data_charges = df[df['charges']<15000]
print("Jumlah sampel sekarang menjadi: {}".format(data_charges.shape[0]))

# Sampel Baru Tagihan Perokok
jml_perokok = data_charges.where(df['smoker']=='yes').value_counts().sum()
print("Jumlah sampel baru perokok: {} orang".format(jml_perokok))

# Sampel Baru Tagihan non perokok
jml_non_perokok = data_charges.where(df['smoker']=='no').value_counts().sum()
print("Jumlah sampel baru non perokok: {} orang".format(jml_non_perokok))

# Tentukan rata-rata charges(tagihan) dari kedua populasi
print("\nRata-rata tagihan pada perokok dan non perokok: ")
mu = data_charges[['charges']].groupby(df['smoker']).mean()
print(mu)
mu1 = mu['charges']['no']
mu2 = mu['charges']['yes']

sns.displot(data_charges, x='charges')
plt.axvline(x=mu1, linestyle='--', c='red', label='mean tagihan non perokok')
plt.axvline(x=mu2, linestyle='--', c='purple', label='mean tagihan perokok')

In [ ]:
# Pisahkan sampel tagihan perokok dengan non-perokok
tagihan_smoker = data_charges['charges'].where(df['smoker']=='yes').dropna()
tagihan_non_smoker = data_charges['charges'].where(df['smoker']=='no').dropna()

**Langkah 2: Hitung Statistik Uji**

Pada kasus ini digunakan statistik Uji T.

**Langkah 3: Menentukan aturan keputusan**

Aturan keputusan didasarkan pada nilai spesifik dari statistik uji (misal, tolak H0 jika Z > 1.645). Karena pada H1 mengandung tanda "<", maka disini dilakukan Lower-tailed test. Dalam uji pihak kiri, aturan keputusan memiliki aturan menolak H0 jika statistik uji lebih kecil dari **nilai kritis**.

**Langkah 4: Menentukan Kritikal Value** 

In [ ]:
print("Varian Tagihan Perokok: {}".format(tagihan_smoker.var()))
print("Varian Tagihan non Perokok: {}".format(tagihan_non_smoker.var()))

In [ ]:
# Langkah 2 s/d 4 diimplementasikan sekaligus
from scipy.stats import ttest_ind

# Lakukan statistik uji T dan hitung p_value
test_stats, p_value = ttest_ind(a=tagihan_smoker, b=tagihan_non_smoker, equal_var=False, alternative='less')
print("Hasil Uji T: {}".format(test_stats))
print("P-value: {}".format(p_value))

**Langkah 5: Membuat Keputusan**

Ada dua bentuk umum yang dapat diambil dari hasil uji hipotesis statistik.
- p-value. 
    - Nilai `P` digunakan dalam pengujian hipotesis untuk membantu memutuskan apakah akan menolak hipotesis nol.
    - Nilai `p-value` adalah bukti terhadap hipotesis nol. Semakin kecil nilai `p-value`, semakin kuat bukti bahwa kita harus menolak hipotesis nol.
- critical value
    - Nilai batas yang digunakan untuk menandai dimulainya suatu wilayah di mana statistik uji, yang diperoleh dalam pengujian hipotesis, tidak mungkin jatuh.
    - Nilai kritis dibandingkan dengan statistik uji yang diperoleh untuk menentukan apakah hipotesis nol harus ditolak atau tidak.
    - Statistik uji < Critical value: Hasil tidak signifikan, gagal menolak H0.
    - Statistik uji >= Critical value: Hasil signifikan, tolak hipotesis H0.

In [ ]:
alpha = 0.05
if p_value > alpha:
    print("Gagal tolak H0")
else:
    print("Tolak H0")

Berdasarkan keputusan yang didapatkan, maka kesimpulan dari pengujian hipotesis yang sudah dilakukan adalah **Tagihan kesehatan perokok lebih tinggi daripada tagihan kesehatan non perokok**.

---
### Hipotesis #4. Tagihan kesehatan dengan BMI diatas 25 lebih tinggi daripada tagihan kesehatan dengan BMI dibawah 25

**Hipotesis**
- H0: Tagihan kesehatan dengan BMI di atas 25 > Tagihan kesehatan dengan BMI di bawah 25.
- H1: Tagihan kesehatan dengan BMI di atas 25 < Tagihan kesehatan dengan BMI di bawah 25.
- $\alpha$: 5%

Kasus ini mirip dengan penyelesaian pada Hipotesis #1, di mana kondisi distribusi data tagihan (`charges`) adalah positiven skew. Maka dari itu kita bisa mengabaikan data outlier untuk kemudian masuk ke statistik uji.

In [ ]:
# Filter data tagihan, gunakan tagihan (charges) < 15K
data_charges = df[df['charges']<15000]
print("Jumlah sampel sekarang menjadi: {}".format(data_charges.shape[0]))

# Sampel Baru Tagihan BMI>25
bmi_g25_tagihan = data_charges.where(df['bmi']>25).value_counts().sum()
print("Jumlah sampel tagihan dengan BMI>25: {} orang".format(bmi_g25_tagihan))

# Sampel Baru Tagihan BMI<25
bmi_l25_tagihan = data_charges.where(df['bmi']<=25).value_counts().sum()
print("Jumlah sampel tagihan dengan BMI<=25: {} orang".format(bmi_l25_tagihan))

# Tentukan rata-rata charges(tagihan) dari kedua populasi
print("\nRata-rata tagihan pada BMI>25 dan BMI<=25: ")
mu = data_charges[['charges']].groupby(df['bmi']>25).mean()
print(mu)
mu1 = mu['charges'][0]
mu2 = mu['charges'][1]

sns.displot(data_charges, x='charges')
plt.axvline(x=mu1, linestyle='--', c='red', label='mean tagihan')
plt.axvline(x=mu2, linestyle='--', c='purple', label='mean tagihan')

In [ ]:
# Pisahkan sampel tagihan dengan BMI> dan BMI<=25
bmi_g25 = data_charges['charges'].where(df['bmi']>25).dropna()
bmi_l25 = data_charges['charges'].where(df['bmi']<=25).dropna()
print("Varian Tagihan BMI>25: {}".format(bmi_g25.var()))
print("Varian Tagihan BMI<=25: {}".format(bmi_l25.var()))
equal_var = True if bmi_g25.var()==bmi_l25.var() else False
print("Apakah varian kedua sampel sama? {}".format(equal_var))

**Lakukan Statistik Uji T dan Hitung p_value**

In [ ]:
alpha = 0.05
test_stats, p_value = ttest_ind(a=bmi_g25, b=bmi_l25, equal_var=equal_var, alternative='less')
print("Hasil Uji T: {}".format(test_stats))
print("P-value: {}".format(p_value))

if p_value > alpha:
    print("Gagal tolak H0")
else:
    print("Tolak H0")

Berdasarkan keputusan yang didapatkan, maka kesimpulan dari pengujian hipotesis yang sudah dilakukan adalah **Tagihan kesehatan orang dengan BMI lebih tinggi dari 25 lebih besar daripada orang yang BMI-nya di bawah 25**.

---
### Hipotesis #5. Tagihan kesehatan laki-laki lebih besar dari perempuan

Hal yang serupa bisa kita lakukan untuk Hipotesis 5.

- H0: Tagihan kesehatan laki-laki > tagihan kesehatan perempuan
- H1: Tagihan kesehatan laki-laki <= tagihan kesehatan perempuan
- $\alpha$: 5%

In [ ]:
# Filter data tagihan, gunakan tagihan (charges) < 15K
data_charges = df[df['charges']<15000]
print("Jumlah sampel sekarang menjadi: {}".format(data_charges.shape[0]))

# Sampel Baru Tagihan Laki-laki
tagihan_laki = data_charges.where(df['sex']=='male').value_counts().sum()
print("Jumlah sampel tagihan pada gender laki-laki: {} orang".format(tagihan_laki))

# Sampel Baru Tagihan Perempuan
tagihan_perempuan = data_charges.where(df['sex']=='female').value_counts().sum()
print("Jumlah sampel tagihan pada gender perempuan: {} orang".format(tagihan_perempuan))

# Tentukan rata-rata charges(tagihan) dari kedua sampel
print("\nRata-rata Tagihan pada Laki-laki dan Perempuan: ")
mu = data_charges[['charges']].groupby(df['sex']).mean()
print(mu)
mu1 = mu['charges']['male']
mu2 = mu['charges']['female']

sns.displot(data_charges, x='charges')
plt.axvline(x=mu1, linestyle='--', c='red', label='mean tagihan')
plt.axvline(x=mu2, linestyle='--', c='purple', label='mean tagihan')

In [ ]:
# Pisahkan sampel tagihan pada gender laki-laki dan perempuan
male = data_charges['charges'].where(df['sex']=='male').dropna()
female = data_charges['charges'].where(df['sex']=='female').dropna()
print("Varian Tagihan Laki-laki: {}".format(male.var()))
print("Varian Tagihan Perempuan: {}".format(female.var()))
equal_var = True if male.var()==female.var() else False
print("Apakah varian kedua sampel sama? {}".format(equal_var))

In [ ]:
# Uji statistik T dan P_value
alpha = 0.05
test_stats, p_value = ttest_ind(a=male, b=female, equal_var=equal_var, alternative='less')
print("Hasil Uji T: {}".format(test_stats))
print("P-value: {}".format(p_value))

if p_value > alpha:
    print("Gagal tolak H0")
else:
    print("Tolak H0")

Berdasarkan keputusan yang didapatkan, maka kesimpulan dari pengujian hipotesis yang sudah dilakukan adalah **Tagihan kesehatan laki-laki tidak lebih besar daripada tagihan kesehatan perempuan**.